In [14]:
import hashlib
import random
import time

class EllipticCurve:
    def __init__(self, a, b, q):
        self.a = a
        self.b = b
        self.q = q

    def is_point_on_curve(self, x, y):
        return (y ** 2 - (x ** 3 + self.a * x + self.b)) % self.q == 0

class Point:
    def __init__(self, curve, x, y):
        self.curve = curve
        self.x = x
        self.y = y

    def __add__(self, other):
        if self.curve != other.curve:
            raise ValueError("Points are not on the same curve")
        if self.x is None:
            return other
        if other.x is None:
            return self
        if self.x == other.x and self.y != other.y:
            return None
        if self == other:
            lam = (3 * self.x ** 2 + self.curve.a) * pow(2 * self.y, -1, self.curve.q)
        else:
            lam = (other.y - self.y) * pow(other.x - self.x, -1, self.curve.q)
        x3 = (lam ** 2 - self.x - other.x) % self.curve.q
        y3 = (lam * (self.x - x3) - self.y) % self.curve.q
        return Point(self.curve, x3, y3)

    def __mul__(self, scalar):
        res = None
        for i in range(scalar.bit_length()):
            if (scalar >> i) & 1:
                res += self * (1 << i)
        return res

class SecureChannel:
    def __init__(self, IoTD, PN):
        self.IoTD = IoTD
        self.PN = PN

    def generate_TS(self):
        # Generate current timestamp
        return int(time.time())

class Registrant:
    def __init__(self, PID_1, ID_R, CH_i, R_i):
        self.PID_1 = PID_1
        self.ID_R = ID_R
        self.CH_i = CH_i
        self.R_i = R_i

class PN:
    def __init__(self):
        self.registry = set()
        self.database = {}

    def authenticate_and_update_parameters(self, registrant):
        # Step 2: Verify validness of registrant
        if registrant.PID_1 in self.registry:
            TS_1 = time.time()  # Current timestamp
            s_2 = random.randint(1, registrant.curve.q - 1)  # Random number s_2
            B_2 = s_2 * public_params['generator']  # Compute B_2

            # Step 3: Compute B_3, B_4, and Auth_1
            s_1 = registrant.B_1 ^ registrant.PID_1 ^ registrant.ID_R
            B_3 = self.database[registrant.PID_1]['CH_i'] + s_1 * B_2
            B_4 = hashlib.sha1((registrant.R_i + str(s_1) + str(TS_1)).encode()).digest()[:20]  # Take first 20 bytes
            Auth_1 = hashlib.sha1((registrant.CH_i + str(B_3) + str(s_1) + str(s_2)).encode()).digest()[:20]

            # Step 4: Send M_2 to registrant
            M_2 = {'TS_1': TS_1, 'B_2': B_2, 'B_3': B_3, 'B_4': B_4, 'Auth_1': Auth_1}
            return M_2
        else:
            return None

class IoTD:
    def __init__(self, MAC_address, curve):
        self.MAC_address = MAC_address
        self.curve = curve

    def generate_PID(self):
        PRNG = hashlib.sha1(self.MAC_address.encode()).digest()
        PID_1 = int.from_bytes(PRNG, byteorder='big')  # Convert bytes to integer
        return PID_1

    def generate_B1(self, PID_1, ID_R):
        s_1 = random.randint(1, self.curve.q - 1)
        ID_R_int = int(ID_R, 16)  # Convert hexadecimal hash string to integer
        B_1 = s_1 ^ PID_1 ^ ID_R_int
        return B_1



def authentication_and_parameters_update_phase(PN_obj, IoTD_obj):
    PID_1 = IoTD_obj.generate_PID()
    ID_R = hashlib.sha1(str(random.random()).encode()).hexdigest()  # Generate a random ID_R
    B_1 = IoTD_obj.generate_B1(PID_1, ID_R)

    # Step 1: IoTD sends M_1 to PN
    registrant = Registrant(PID_1, ID_R, IoTD_obj.MAC_address, B_1)
    M_1 = {'PID_1': registrant.PID_1, 'B_1': B_1}
    M_2 = PN_obj.authenticate_and_update_parameters(registrant)
    if M_2:
        print("Authentication successful! M_2:", M_2)
    else:
        print("Authentication failed. Invalid PID_1.")


# Example usage
PN_obj = PN()
curve = EllipticCurve(0, 7, 23)  # Define your elliptic curve here
IoTD_obj = IoTD("00:11:22:33:44:55", curve)  # Example MAC address
authentication_and_parameters_update_phase(PN_obj, IoTD_obj)


Authentication failed. Invalid PID_1.
